In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

import cv2
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
from tqdm import tqdm


In [13]:
def read_xray(path, voi_lut=True, fix_monochrome=True, use_8bit=True, rescale_time=None):
    dicom = pydicom.read_file(path)
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
    
    data = data.astype(np.float64)
    if rescale_time:
        data = cv2.resize(data, (data.shape[1] // rescale_time, data.shape[0] // rescale_time), interpolation=cv2.INTER_CUBIC)

    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
    data = data - np.min(data)
    data = data / np.max(data)
    if use_8bit:
        data = (data * 255).astype(np.uint8)
    else:
        data = (data * 65535).astype(np.uint16)
    return data

In [14]:
data_path = "/workspace/nabang1010/STEAM/LeGiaBach_STEAM/DATA/dataset/train"
data_image = "/workspace/nabang1010/STEAM/LeGiaBach_STEAM/DATA/dataset/images"

In [16]:
for i in os.listdir(data_path):
    # print(i)
    path = os.path.join(data_path, i)
    img_8bit = read_xray(path)
    cv2.imwrite(os.path.join(data_image , i[ :-5] + 'png'), img_8bit)
    # break


KeyboardInterrupt: 

In [ ]:
# 3D convolution layer with dimensions CxDxHxW = 16x21x63x63 
# This convolutional layer has 32 neurons with a receptive field size of 7x7x7
import torch
import torch.nn as nn

class Conv3d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True):
        super(Conv3d, self).__init__()
        self.conv3d = nn.Conv3d(in_channels, out_channels, kernel_size, stride, padding, dilation, groups, bias)
        self.relu = nn.ReLU(inplace=True)
        self.bn = nn.BatchNorm3d(out_channels)
        
    def forward(self, x):
        x = self.conv3d(x)
        x = self.relu(x)
        x = self.bn(x)
        return x

# 3D max pooling layer with dimensions CxDxHxW = 16x21x63x63
# This max pooling layer has a receptive field size of 2x2x2
    
    